In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-04 05:28:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-12-04 05:28:47 (11.2 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-Vine").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [ ]:
from pyspark.sql.functions import to_date, count

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

In [ ]:
# Step 1 -- Filter the dataset to create a new DataFrame where the total votes count is greater than or equal to 20
vine_df1 = vine_df.filter(vine_df["total_votes"] >= 20)
vine_df1.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
| RUNB838XKMB59|          1|            8|         23|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
|R2WFUSIPJ3X0FR|          1|            9|         53|   N|                N|
| RONWXF0J85CRR|          5|           42|         50|   N|     

In [ ]:
# Step 2 -- Filter the new dataset above to create a new DataFrame to retrieve all rows where 
# the number of "helpful_votes" divided by the number of "total_votes" is greater than or equal to 50% 
vine_df2 = vine_df1.filter(vine_df1["helpful_votes"] / vine_df1["total_votes"] >= 0.5)
vine_df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [ ]:
# Step 3 -- Filter the dataset from above to create a new DataFrame where a review was not written as part of the Vine program (unpaid)
# Please note -- this order is reversed from what is written in the Deliverable instructions because I first filtered for "N"
vine_dfN = vine_df2.filter(vine_df2["vine"] == "N")
vine_dfN.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|
| RSU37FK0W4DPE|          4|           26|         28|   N|                N|
|R22H100N6EUD0Y|          5|           30|         32|   N|     

In [ ]:
# Step 4 -- Filter the dataset from above to create a new DataFrame where a review was written as part of the Vine program (paid) 
vine_dfY = vine_df2.filter(vine_df2["vine"] == "Y")
vine_dfY.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PBQ665MBL8CP|          3|           66|         72|   Y|                N|
|R3EMD30K7MNOZ0|          2|           20|         25|   Y|                N|
|R2GF3ZLO2B0Y71|          5|          236|        243|   Y|                N|
|R1F8T2Z7U3N6HL|          3|           11|         20|   Y|                N|
| R5QTOT1GVR8RG|          4|           16|         20|   Y|                N|
|R1KAQEESF0NUGV|          4|           27|         29|   Y|                N|
|R1621B9AMVWI25|          2|           27|         29|   Y|                N|
|R3FVGQVNBAZL8E|          3|           37|         44|   Y|                N|
|R26ATTA765WHVX|          5|           21|         23|   Y|                N|
| RMZYPFKES1ZK7|          4|           29|         33|   Y|     

In [ ]:
# Total Number of Non-Vine reviews
total_NV_reviews = vine_dfN.agg({"review_id": "count"})
total_NV_reviews.show()

+----------------+
|count(review_id)|
+----------------+
|          151400|
+----------------+



In [ ]:
# Total number of Vine reviews
total_V_reviews = vine_dfY.agg({"review_id": "count"})
total_V_reviews.show()

+----------------+
|count(review_id)|
+----------------+
|              49|
+----------------+



In [ ]:
# Number of 5 star Non-Vine reviews 
fivestar_NV = vine_dfN.filter(vine_dfN["star_rating"] == 5).agg({"star_rating": "count"})
fivestar_NV.show()

+------------------+
|count(star_rating)|
+------------------+
|             78061|
+------------------+



In [ ]:
# Number of 5 star Vine reviews 
fivestar_V = vine_dfY.filter(vine_dfY["star_rating"] == 5).agg({"star_rating": "count"})
fivestar_V.show()


+------------------+
|count(star_rating)|
+------------------+
|                 9|
+------------------+



In [ ]:
# Create a DataFrame to compile summary data for final analysis
amazon_vine_reviews = spark.createDataFrame([
                                   (151400, 49, 78061, 9)
], ["total_NV_reviews", "total_V_reviews", "fivestar_NV", "fivestar_V"])
amazon_vine_reviews.show()


+----------------+---------------+-----------+----------+
|total_NV_reviews|total_V_reviews|fivestar_NV|fivestar_V|
+----------------+---------------+-----------+----------+
|          151400|             49|      78061|         9|
+----------------+---------------+-----------+----------+



In [ ]:
# Calculate the percentage of Non-Vine 5 star reviews and add to the summary table
percentage_NV = amazon_vine_reviews.withColumn('percentage_NV', amazon_vine_reviews['fivestar_NV'] / amazon_vine_reviews['total_NV_reviews'] * 100)
percentage_NV.show()


+----------------+---------------+-----------+----------+-----------------+
|total_NV_reviews|total_V_reviews|fivestar_NV|fivestar_V|    percentage_NV|
+----------------+---------------+-----------+----------+-----------------+
|          151400|             49|      78061|         9|51.55944517833554|
+----------------+---------------+-----------+----------+-----------------+



In [ ]:
# Calculate the percentage of Vine 5 star reviews and add to the summary table
final_summary_reviews = percentage_NV.withColumn('percentage_V', percentage_NV['fivestar_V'] / percentage_NV['total_V_reviews'] * 100).show()


+----------------+---------------+-----------+----------+-----------------+------------------+
|total_NV_reviews|total_V_reviews|fivestar_NV|fivestar_V|    percentage_NV|      percentage_V|
+----------------+---------------+-----------+----------+-----------------+------------------+
|          151400|             49|      78061|         9|51.55944517833554|18.367346938775512|
+----------------+---------------+-----------+----------+-----------------+------------------+

